In [ ]:
# Import libraries and modules
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# Load the image
image_path = r"..\test_images\normal_pressure.png"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)


In [ ]:
# Invert colors if needed (to make the curve stand out)
_, thresh = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY_INV)

In [ ]:
# Edge detection (Canny)
edges = cv2.Canny(thresh, 50, 150)

In [ ]:
# Apply thresholding to highlight the waveform
_, binary = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY_INV)


In [ ]:
# Automatically determine Canny thresholds
median_intensity = np.median(binary)
low_threshold = int(max(0, 0.66 * median_intensity))  # Lower threshold
high_threshold = int(min(255, 1.33 * median_intensity))  # Higher threshold


In [ ]:
# Apply edge detection (Canny) with dynamic thresholds
edges = cv2.Canny(binary, low_threshold, high_threshold)


In [ ]:
# Show the edge-detected image for reference
plt.figure(figsize=(8, 5))
plt.imshow(binary, cmap="gray")
plt.title(f"Canny Edge Detection (Low: {low_threshold}, High: {high_threshold})")
plt.axis("off")
plt.show()


* The code works fine up to this point. The edges are found correctly. You need to optimise the process of obtaining pixels from detected edges (code below)

---

In [ ]:
# Find contours of the waveform
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)


In [ ]:
# Ensure the largest contour is the waveform (ignore small objects)
contours = sorted(contours, key=cv2.contourArea, reverse=True)
largest_contour = contours[0]  # Pick the biggest detected shape


In [ ]:
# Convert contour to array of points
x_pixels = largest_contour[:, 0, 0]  # Horizontal (time axis)
y_pixels = largest_contour[:, 0, 1]  # Vertical (pressure axis)


In [ ]:
# Ensure x_pixels are strictly increasing (avoid disorder in time values)
sorted_indices = np.argsort(x_pixels)
x_pixels = x_pixels[sorted_indices]
y_pixels = y_pixels[sorted_indices]


In [ ]:
# Remove duplicate x values to ensure smooth plotting
unique_x, unique_indices = np.unique(x_pixels, return_index=True)
x_pixels = unique_x
y_pixels = y_pixels[unique_indices]


In [ ]:
# Define known axis values from the image (based on labels)
x_min, x_max = 0, 4  # Time range in seconds
y_min, y_max = -10, 10  # Pressure range in cmH2O


In [ ]:
# Function to normalize pixel coordinates to real-world values
def normalize(value, min_pixel, max_pixel, min_real, max_real):
    return min_real + (value - min_pixel) / (max_pixel - min_pixel) * (max_real - min_real)


In [ ]:
# Scale x and y values correctly
x_scaled = normalize(x_pixels, np.min(x_pixels), np.max(x_pixels), x_min, x_max)
y_scaled = normalize(y_pixels, np.max(y_pixels), np.min(y_pixels), y_min, y_max)  # Flip y-axis correctly


In [ ]:
# Create a DataFrame with extracted values
data = pd.DataFrame({'Time (s)': x_scaled, 'Pressure (cmH2O)': y_scaled})
# Display the first few rows
print(data.head())


In [ ]:
# Plot the extracted waveform
plt.figure(figsize=(8, 5))
plt.plot(data["Time (s)"], data["Pressure (cmH2O)"], marker="o", linestyle="-", color="b")
plt.xlabel("Time (s)")
plt.ylabel("Pressure (cmH2O)")
plt.title("Extracted Normal Pressure Waveform")
plt.grid(True)
plt.show()
